# MVP1
### Kellen O'Connor
## Predicting Peat Depth in Minnesota

#### Clipping County DEM to Watershed boundary


In [4]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd

# Path to the dem, huc8 shapefile, and output folder
dem_path = "5543/scott/dem_3m_scott.tif"
huc_path = "5543/hucscott/huc_scott.shp"
output_folder = "clipped_rasters"
output_tif = os.path.join(output_folder, "dem_clipped.tif")

# Load watershed shapefile
watershed = gpd.read_file(huc_path)

# Open the dem
with rasterio.open(dem_path) as src:
    # Project shapefile to dem because the coordinate systems were off
    if watershed.crs != src.crs:
        watershed = watershed.to_crs(src.crs)
    
    # Get geometry
    geoms = [feature.__geo_interface__ for feature in watershed.geometry]

    # Set nodata value so the raster is in shape of the watershed
    nodata_value = -9999999

    # Clip the raster
    out_image, out_transform = mask(
        dataset=src,
        shapes=geoms,
        crop=True,
        nodata=nodata_value
    )

    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
        "nodata": nodata_value
    })

# Make the output tif file
with rasterio.open(output_tif, "w", **out_meta) as dest:
    dest.write(out_image)

print(f"Bless it worked")


Bless it worked


This code is a MVP of my project because it is the first step in the process of curating the dems to be used in making the machine learning model. Looking forward, attemping to complete all of the specs I listed in the MVP document for this project might be ambitiuos to attempt during just a semester. I will conitnue to build on this script so it can eventually run this process for dems of the entire state. This will be done using MSI and submitting a slurm job 